# Hot-spot factor calibration

## Introduction

In some cases, when modelling a power transformer, the hot-spot factor is not known. 
Then, hot-spot factor calibration is used to determine it.
This is done based on a continuous nominal load with a duration of one week, a specified constant ambient temperature,
and a hot-spot temperature limit, the hot-spot factor is calibrated to get as close as
possible to 100% nominal load while staying within the 'hot_spot_factor_min' and 'hot_spot_factor_max' bounds.

This example shows how to do this for a basic Power transformer.

## Setup Inputprofile and Transformer

In [ ]:
import pandas as pd

from transformer_thermal_model.cooler import CoolerType
from transformer_thermal_model.model import Model
from transformer_thermal_model.schemas import InputProfile, UserTransformerSpecifications
from transformer_thermal_model.transformer import PowerTransformer

# In this example the model is used to calculate the transformer temperature based on a load and ambient
# profile with a period of one week. Any duration can be chosen preferably with timestamps with an interval of
# 15 minute or lower. Larger timesteps will result in incorrect results but it *is* possible to calculate with them.
one_week = 4 * 24 * 7
datetime_index = pd.date_range("2020-01-01", periods=one_week, freq="15min")

# For the load (in A) and ambient temperature (in C) arbitrary constants profiles are chosen.
# It is also possible to use a realistic profile.
nominal_load = 100
load_points = pd.Series([nominal_load] * one_week, index=datetime_index)
ambient_temp = 21
temperature_points = pd.Series([ambient_temp] * one_week, index=datetime_index)

# Create an input object with the profiles
profile_input = InputProfile.create(
    datetime_index=datetime_index, load_profile=load_points, ambient_temperature_profile=temperature_points
)

# Initialise a power transformer with cooling type ONAF and, besides the mandatory user specifications, default values.
tr_specs = UserTransformerSpecifications(
    load_loss=1000,  # Transformer load loss [W]
    nom_load_sec_side=1500,  # Transformer nominal current secondary side [A]
    no_load_loss=200,  # Transformer no-load loss [W]
    amb_temp_surcharge=20,  # Ambient temperature surcharge [K]
)
transformer = PowerTransformer(user_specs=tr_specs, cooling_type=CoolerType.ONAF)

## Determining the hot-spot factor

The temperature in the model is modelled using the hot-spot factor (link will follow). This is a value between 1.1 and
1.3. 

When unknown, we have also provided a functionality to calibrate this value **for power transformers only**. 
When you have the rest of the specifications of your transformer:

In [2]:
from transformer_thermal_model.cooler import CoolerType
from transformer_thermal_model.hot_spot_calibration import calibrate_hotspot_factor
from transformer_thermal_model.schemas import UserTransformerSpecifications
from transformer_thermal_model.transformer import PowerTransformer

tr_specs = UserTransformerSpecifications(
    load_loss=1000,  # Transformer load loss [W]
    nom_load_sec_side=1500,  # Transformer nominal current secondary side [A]
    no_load_loss=200,  # Transformer no-load loss [W]
    amb_temp_surcharge=20,  # Ambient temperature surcharge [K]
)
uncalibrated_transformer = PowerTransformer(user_specs=tr_specs, cooling_type=CoolerType.ONAF)
calibrated_trafo = calibrate_hotspot_factor(
    uncalibrated_transformer=uncalibrated_transformer,
    ambient_temp=20.0,
    hot_spot_limit=98,  # in most cases a hot-spot temperature limit of 98 can be used
    hot_spot_factor_min=1.1,
    hot_spot_factor_max=1.3,
)


The new hot-spot factor is available via `calibrated_trafo.specs.hot_spot_fac`, and from now on used in the thermal model.



In [3]:
print(calibrated_trafo.specs.hot_spot_fac)

1.1


The calibrated transformer is now ready to be used in the Model

In [ ]:
model = Model(temperature_profile=profile_input, transformer=calibrated_trafo)

results = model.run()

# Get the results as pd.Series, with the same datetime_index as your input.
top_oil_temp_profile = results.top_oil_temp_profile
hot_spot_temp_profile = results.hot_spot_temp_profile